In [ ]:
import re
#import nlppt as nl
import nltk
from tqdm import tqdm
from nltk.corpus import stopwords
from time import time
from nltk.tokenize import sent_tokenize,word_tokenize
import pickle
import numpy as np

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
import pandas as pd

df = pd.read_csv("https://raw.githubusercontent.com/codebasics/deep-learning-keras-tf-tutorial/master/47_BERT_text_classification/spam.csv")
df.head(5)

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [ ]:
def preprocessing(data):
    
    data = data.fillna("")
    data.isnull().any()

    #start = time()
    punctuation = ["/",",",".",";",":","(",")","-","[","]","#","£","?","&","ü","!"]
    data["text"] = data.Message
    data["text"] = [word_tokenize(s.lower()) for s in data["text"]]
    data["text"] = [[w for w in s if w not in stopwords.words("english")] for s in data["text"]]
    data["text"] = [[w for w in s if w not in punctuation] for s in data["text"]]
    data["text"] = [[w for w in s if not(re.match(r"\d",w))] for s in data["text"]]
    data["text"] = [[w[:-1] if w[-1] in punctuation else w for w in s] for s in data["text"]]
    #print("Time: %.4f"%(time() - start) )
    
    out = "/content"
    # Saving preprocessed cvs
    f = open(out+"/phrases.series",'wb')
    cvs_phrases = data["text"]
    pickle.dump(cvs_phrases,f)
    #print("CVs saved!")
    
    return data

In [ ]:
df_processed = preprocessing(df)

In [ ]:
df_processed= df_processed.drop('Message',axis=1)
df_processed.columns

Index(['Category', 'text'], dtype='object')

In [ ]:
df_processed['Category'].value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [ ]:
df_spam = df_processed[df_processed['Category']=='spam']
df_spam.shape

(747, 2)

In [ ]:
df_ham = df_processed[df_processed['Category']=='ham']
df_ham.shape

(4825, 2)

In [ ]:
df_ham_upsampled = df_ham.sample(df_ham.shape[0])
df_ham_upsampled.shape

(747, 2)

In [ ]:
df_balanced = pd.concat([df_ham_upsampled, df_spam])
df_balanced.shape

(1494, 2)

In [ ]:
df_balanced['Category']=df_balanced['Category'].apply(lambda x: 1 if x=='spam' else 0)

In [ ]:
from sklearn.model_selection import train_test_split
# Train Test Split Function
def split_train_test(df_balanced, test_size=0.3, shuffle_state=True):
    X_train, X_test, Y_train, Y_test = train_test_split(df_balanced[['text']], 
                                                        df_balanced['Category'], 
                                                        shuffle=shuffle_state,
                                                        test_size=test_size, 
                                                        random_state=15)
    return X_train, X_test, Y_train, Y_test

In [ ]:
X_train, X_test, Y_train, Y_test= split_train_test(df_balanced)

In [ ]:
def dependent_data_format(data):
  samples = []
  for row in data:
    samples.append(row)
  y_train  = np.array(samples)
  return y_train

In [ ]:
y_train = dependent_data_format(Y_train)

In [ ]:
y_test = dependent_data_format(Y_test)

In [ ]:
def independent_data_format(data):
  lists=[]
  for text in data['text']:
    string = ""  
    for word in text:
      string+=  word + " " 
    string = string.rstrip()
    lists.append(string)
  return lists

In [ ]:
x_train = independent_data_format(X_train)

In [ ]:
x_test =  independent_data_format(X_test)

In [ ]:
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LinearSVC()),
                     ])

text_clf.fit(x_train, y_train)


predicted = text_clf.predict(x_test)

print(metrics.classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95       218
           1       0.98      0.92      0.95       231

    accuracy                           0.95       449
   macro avg       0.95      0.95      0.95       449
weighted avg       0.95      0.95      0.95       449



In [ ]:
inf = text_clf.predict(["order this type of phone and you will win a hamper"])[0]

In [ ]:
inf

0

In [ ]:
inf = text_clf.predict(["this is a formal invitation for the tech festival that is going to take place in banglore"])[0]

In [ ]:
inf

0

In [ ]:
from sklearn.svm import LinearSVC
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', LinearSVC()),
                     ])

text_clf.fit(x_train, y_train)


predicted = text_clf.predict(x_test)

print(metrics.classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.92      0.98      0.95       218
           1       0.98      0.92      0.95       231

    accuracy                           0.95       449
   macro avg       0.95      0.95      0.95       449
weighted avg       0.95      0.95      0.95       449



In [ ]:
from keras.layers import  Dropout, Dense
from keras.models import Sequential
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn import metrics


def TFIDF(X_train, X_test,MAX_NB_WORDS=75000):
  #print(X_test)
  vectorizer_x = TfidfVectorizer(max_features=MAX_NB_WORDS, use_idf=True)
  pickle.dump(vectorizer_x, open("new_vectorizer.pickle", "wb"))
  #pickle.dump(vectorizer_x.vocabulary_,open("feature.pkl","wb"))
  X_train = vectorizer_x.fit_transform(X_train).toarray()
  X_test = vectorizer_x.transform(X_test).toarray()
  print(vectorizer_x)
  print("tf-idf with",str(np.array(X_train).shape[1]),"features")
  return (X_train,X_test)


def Build_Model_DNN_Text(shape, nClasses, dropout=0.5):
    """
    buildModel_DNN_Tex(shape, nClasses,dropout)
    Build Deep neural networks Model for text classification
    Shape is input feature space
    nClasses is number of classes
    """
    model = Sequential()
    node = 512 # number of nodes
    nLayers = 4 # number of  hidden layer

    model.add(Dense(node,input_dim=shape,activation='relu'))
    model.add(Dropout(dropout))
    for i in range(0,nLayers):
        model.add(Dense(node,input_dim=node,activation='relu'))
        model.add(Dropout(dropout))
    model.add(Dense(nClasses, activation='softmax'))

    model.compile(loss='sparse_categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model



X_train_tfidf,X_test_tfidf = TFIDF(x_train,x_test)


model_DNN = Build_Model_DNN_Text(X_train_tfidf.shape[1], 20)
model_DNN.summary()
exit(1)
model_DNN.fit(X_train_tfidf, y_train,
                              validation_data=(X_test_tfidf, y_test),
                              epochs=10,
                              batch_size=128,
                              verbose=2)

predicted = model_DNN.predict(X_test_tfidf)
predicted

y_predicted_labels = [np.argmax(i) for i in predicted]
print(metrics.classification_report(y_test, y_predicted_labels))

TfidfVectorizer(max_features=75000)
tf-idf with 3113 features
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_30 (Dense)            (None, 512)               1594368   
                                                                 
 dropout_25 (Dropout)        (None, 512)               0         
                                                                 
 dense_31 (Dense)            (None, 512)               262656    
                                                                 
 dropout_26 (Dropout)        (None, 512)               0         
                                                                 
 dense_32 (Dense)            (None, 512)               262656    
                                                                 
 dropout_27 (Dropout)        (None, 512)               0         
                                                          

In [ ]:
vectorizer = pickle.load(open("new_vectorizer.pickle", "rb"))
X_test = vectorizer.transform(x_test).toarray() # getting error here

NotFittedError: ignored

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

my_data = ["hello how are you", "hello who are you", "i am not you"]

tf = TfidfVectorizer(use_idf=True)
tf.fit_transform(my_data)

idf = tf.idf_ 
idf


array([1.69314718, 1.28768207, 1.28768207, 1.69314718, 1.69314718,
       1.69314718, 1.        ])

In [ ]:
# If you want to get the idf value for a particular word, here "hello"    
tf.idf_[tf.vocabulary_["hello"]]

1.2876820724517808